In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import requests
from session import SESSION
from common import print_problem, get_problem_input, submit_answer, neighbors
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from pprint import pprint
from functools import lru_cache
import numpy as np
from enum import Enum, IntEnum
from string import ascii_lowercase, ascii_uppercase

In [5]:
DAY = 8

In [6]:
raw_data, data = get_problem_input(DAY)

Raw Data:
'10002111211020231202201033020431204000011101214344514222141424022024044233204001'
Split Data:
['100021112110202312022010330204312040000111012143445142221414240220240442332040010320133120230011020',
 '111110201332323210211143123214321343332124211413115514155115511033421001222101204330001300333011010',
 '222100123300231230122203432310224441551434231352112532354252244334410042212441233243220102033110020',
 '212210311310102334321243104133012535144151532555155341325352512544453402400411340202133231000102020',
 '112002102322001003304233302040124544411225523533534235113353522233535550432232202401221022110311110',
 '112102131331201432320312233124434232544144233241123334112232531521542551332434224211234133132330300',
 '201212231001314001232020011521543445152251413225334151155332443144315433444321403022301110231133320',
 '222200212012234021440323522251222114255435321245463652634454445441313331334221403314121340322201323',
 '0111013130301240340110531324113523131354322652235646445634355

In [7]:
test_data = [
"30373",
"25512",
"65332",
"33549",
"35390",
]

In [8]:
def make_np(data):
    return np.array([list(map(int, line)) for line in data])

In [9]:
def find_visibles(str_data):
    np_data = make_np(str_data)
    height, width = np_data.shape

    lr_data = np.fliplr(np_data)
    ud_data = np.flipud(np_data)
    
    left = np.maximum.accumulate(np_data, axis=1)
    top = np.maximum.accumulate(np_data, axis=0)
    right = np.maximum.accumulate(lr_data, axis=1)
    down = np.maximum.accumulate(ud_data, axis=0)
    
    l_visible = np.concatenate((np.full((height, 1), True), left[...,1:] > left[...,:-1]),axis=1)
    t_visible = np.concatenate((np.full((1, width), True), top[1:,...] > top[:-1,...]),axis=0)
    r_visible = np.fliplr(np.concatenate((np.full((height, 1), True), right[...,1:] > right[...,:-1]),axis=1))
    d_visible = np.flipud(np.concatenate((np.full((1, width), True), down[1:,...] > down[:-1,...]),axis=0))
    
    output = np.bitwise_or(l_visible, t_visible)
    output = np.bitwise_or(output, r_visible)
    output = np.bitwise_or(output, d_visible)
    
    return np.count_nonzero(output)

In [10]:
first = find_visibles(data)

In [42]:
submit_answer(DAY, 1, first)

1796
("That's the right answer!  You are one gold star closer to collecting enough "
 'star fruit. [Continue to Part Two]')


True

In [11]:
def calc_scores(str_data):
    np_data = make_np(str_data)
 
    def iterate_trees(input_arr):
        o_height, o_width = input_arr.shape
        top_views = np.zeros((o_height, o_width))

        for h in range(o_height-1):
            rest = input_arr[h+1:]
            height, width = rest.shape
            start = np.tile(input_arr[h],(height,1))

            comp = rest < start
            acc = np.logical_and.accumulate(comp)
            sum_ints = np.sum(acc.astype(int), axis=0)
                
            a = np.empty(width)
            a.fill(height)

            top_views[h] = sum_ints + np.ones((1, width))
            top_views[h] = np.minimum(top_views[h],a)

        return top_views
        
    t = iterate_trees(np_data)
    d = np.flipud(iterate_trees(np.flipud(np_data)))
    l = iterate_trees(np_data.T).T
    r = np.flipud(iterate_trees(np.flipud(np_data.T))).T
    
    x = np.multiply(t, d)
    x = np.multiply(x, l)
    x = np.multiply(x, r)
    
    return x

In [12]:
second = int(np.max(calc_scores(data)))

In [13]:
second

288120

In [509]:
submit_answer(DAY, 2, second)

288120
("That's the right answer!  You are one gold star closer to collecting enough "
 'star fruit.You have completed Day 8! You can [Shareon\n'
 '  Twitter\n'
 'Mastodon] this victory or [Return to Your Advent Calendar].')


True